In [1]:
import numpy as np
import arcode
import codec
import struct
import os
import matplotlib.pyplot as plt
import matplotlib as mpl
import skimage
from PIL import Image

In [2]:
def show_data(x_in, pre_in):
    mean = np.mean(x_in)
    std = np.std(x_in)
    Max = np.max(x_in)
    Min = np.min(x_in)
    dtype = x_in.dtype
    
    print(pre_in + ' shape: ' + str(x_in.shape) + ' , mean = ' + str(mean) 
          + ' , std = ' + str(std) + ' , Max = ' + str(Max) + ' , Min = ' + str(Min) 
          + ' , type=' + str(dtype))

In [3]:
def oc_fname(i_in):
    rname = '/f00' + str(i_in) + '.png'
    if i_in >9: rname = '/f0' + str(i_in) + '.png'
    if i_in >99: rname = '/f' + str(i_in) + '.png'
        
    return rname

In [4]:
def read_COMPINT(prefC, cname):
    o4=np.zeros([1, nx, ny, ncomp])
    for k in range(ncomp):
        inname = prefC + str(k) + cname
        #print("  read_COMPINT: " + inname)
        tim=Image.open(inname)
        tima=np.array(tim)
        o4[0,:,:,k] = tima
                
    return o4

In [5]:
def ar_code(x_in, oname):
    img = x_in.flatten()
    olen = len(img)
    img = img.reshape(olen,1,1)
    
    bits = codec.bitstream(dat=img)
    bits.h = img.shape[0]
    bits.w = img.shape[1]
    bits.c = img.shape[2]
    entro_coder = codec.entropy_codec(Mode='Arith')
    entro_coder.encode(bits, oname)
    
    size1= os.path.getsize(oname + 'dat')
    size2= os.path.getsize(oname + 'arith')

    #olen, size1, size2 in unit of Byte (assuming input is 8bit int)
    return olen, size1, size2

In [6]:
def ar_decode(aname, dname):
    ar = arcode.ArithmeticCode(False)
    ar.decode_file(aname, dname)

    file_bin = open(dname,'rb')
    s = file_bin.read()
    H,W,C = struct.unpack('3i',s[:12])
    #print('H= ' + str(H) + ' ; W= ' + str(W) + ' ; C= ' + str(C))
    
    bytes=[]
    for i in range(H*W*C):
        bytes.append(struct.unpack('b',s[12+i:12+i+1]))
    file_bin.close()
    bin_data = np.array(bytes).reshape(H,W,C)
    bin_data = bin_data.flatten()
    
    return bin_data

In [40]:
'''
pref = '/home/saberbud/Programs/xy_TF_sectioned_compression/Kodak_Bc_884/COMPINT'
nx=256
ny=384
ncomp=4
'''


pref = '/home/saberbud/Programs/xy_TF_sectioned_compression/Kodak_Rc_16164/COMPINT'
nx=128
ny=192
ncomp=4


'''
pref = '/home/saberbud/Programs/xy_TF_sectioned_compression/Kodak_Bc_32324/COMPINT'
nx=64
ny=96
ncomp=4
'''

print(pref)

/home/saberbud/Programs/xy_TF_sectioned_compression/Kodak_Rc_16164/COMPINT


In [41]:
i_f = 7
t_C = read_COMPINT(pref, oc_fname(i_f))

show_data(t_C, 't_C')
for i in range(4):
    show_data(t_C[...,i], 't_C_'+str(i))

t_C shape: (1, 128, 192, 4) , mean = 130.96996053059897 , std = 30.665196750095618 , Max = 198.0 , Min = 62.0 , type=float64
t_C_0 shape: (1, 128, 192) , mean = 99.02506510416667 , std = 13.131280367287234 , Max = 137.0 , Min = 62.0 , type=float64
t_C_1 shape: (1, 128, 192) , mean = 161.95674641927084 , std = 14.73868383975567 , Max = 194.0 , Min = 121.0 , type=float64
t_C_2 shape: (1, 128, 192) , mean = 107.61653645833333 , std = 9.488630462609578 , Max = 144.0 , Min = 65.0 , type=float64
t_C_3 shape: (1, 128, 192) , mean = 155.281494140625 , std = 12.830955051482675 , Max = 198.0 , Min = 108.0 , type=float64


In [42]:
idc = 3
data_t = t_C[0,:,:,idc]
data_t = data_t - 128.0
data_t = data_t.astype(np.int8)
show_data(data_t, 'data_t')

data_t shape: (128, 192) , mean = 27.281494140625 , std = 12.830955051482675 , Max = 70 , Min = -20 , type=int8


In [43]:
oname = 'fAc'
olen, size1, size2 = ar_code(data_t, oname)

print('olen' + str(olen))
print('size1' + str(size1))
print('size2' + str(size2))

ratio1 = float(size1)/float(olen)
ratio2 = float(size2)/float(olen)

print('ratio1= ' + str(ratio1))
print('ratio2= ' + str(ratio2))

olen24576
size124588
size216423
ratio1= 1.00048828125
ratio2= 0.668253580729


In [44]:
aname = oname + 'arith'
dname = 'dAc.dat'
dcode = ar_decode(aname, dname)

dlen = len(dcode)
print('dlen= ' + str(dlen))

mse = np.mean(np.square(dcode-data_t.flatten()))
print('mse = ' + str(mse))

dlen= 24576
mse = 0.0


In [45]:
fs = 1; fe = 25
nf = fe-fs
size1s = np.zeros([nf,4])
size2s = np.zeros([nf,4])
olens = np.zeros([nf,4])
d_lens = np.ones([nf,4])
mses = np.zeros([nf,4])

for i in range(fs,fe):
    print(i)
    t_C = read_COMPINT(pref, oc_fname(i))
    show_data(t_C, '  t_C')
    
    for idc in range(4):
        data_t = t_C[0,:,:,idc]
        data_t = data_t - 128.0
        data_t = data_t.astype(np.int8)
        show_data(data_t, ' channel'+str(idc))
        olen, size1, size2 = ar_code(data_t, oname)

        print('   olen: ' + str(olen) + ' Bytes')
        print('   size1: ' + str(size1) + ' Bytes')
        print('   size2: ' + str(size2) + ' Bytes')

        ratio1 = float(size1)/float(olen)
        ratio2 = float(size2)/float(olen)

        print('ratio1= ' + str(ratio1))
        print('ratio2= ' + str(ratio2))
        
        dcode = ar_decode(aname, dname)
        dlen = len(dcode)
        print('dlen= ' + str(dlen))

        idx = i-fs
        
        if dlen == olen:
            d_lens[idx,idc] = 0
            mse = np.mean(np.square(dcode-data_t.flatten()))
            mses[idx,idc] = mse
            print('mse = ' + str(mse))
        if dlen != olen:
            d_lens[idx,idc] = 1
            mses[idx,idc] = 99.99
            print('!Length differ!')
        
        size1s[idx,idc] = size1
        size2s[idx,idc] = size2
        olens[idx,idc] = olen
        print('')
    
    print('\n')

1
  t_C shape: (1, 128, 192, 4) , mean = 130.8849894205729 , std = 31.35463791515664 , Max = 210.0 , Min = 47.0 , type=float64
 channel0 shape: (128, 192) , mean = -29.6783447265625 , std = 13.522581174540939 , Max = 11 , Min = -81 , type=int8
   olen: 24576 Bytes
   size1: 24588 Bytes
   size2: 17205 Bytes
ratio1= 1.00048828125
ratio2= 0.700073242188
dlen= 24576
mse = 0.0

 channel1 shape: (128, 192) , mean = 34.763997395833336 , std = 14.183485896346788 , Max = 70 , Min = -5 , type=int8
   olen: 24576 Bytes
   size1: 24588 Bytes
   size2: 17165 Bytes
ratio1= 1.00048828125
ratio2= 0.698445638021
dlen= 24576
mse = 0.0

 channel2 shape: (128, 192) , mean = -21.240071614583332 , std = 10.53012090856691 , Max = 16 , Min = -79 , type=int8
   olen: 24576 Bytes
   size1: 24588 Bytes
   size2: 16202 Bytes
ratio1= 1.00048828125
ratio2= 0.659261067708
dlen= 24576
mse = 0.0

 channel3 shape: (128, 192) , mean = 27.694376627604168 , std = 12.781713938913848 , Max = 82 , Min = -16 , type=int8
   o

dlen= 24576
mse = 0.0

 channel1 shape: (128, 192) , mean = 43.34912109375 , std = 16.202530502061464 , Max = 91 , Min = -3 , type=int8
   olen: 24576 Bytes
   size1: 24588 Bytes
   size2: 18604 Bytes
ratio1= 1.00048828125
ratio2= 0.756998697917
dlen= 24576
mse = 0.0

 channel2 shape: (128, 192) , mean = -26.207600911458332 , std = 11.824603209168163 , Max = 10 , Min = -70 , type=int8
   olen: 24576 Bytes
   size1: 24588 Bytes
   size2: 17363 Bytes
ratio1= 1.00048828125
ratio2= 0.706502278646
dlen= 24576
mse = 0.0

 channel3 shape: (128, 192) , mean = 34.766682942708336 , std = 14.338892832293052 , Max = 81 , Min = -10 , type=int8
   olen: 24576 Bytes
   size1: 24588 Bytes
   size2: 18130 Bytes
ratio1= 1.00048828125
ratio2= 0.737711588542
dlen= 24576
mse = 0.0



9
  t_C shape: (1, 128, 192, 4) , mean = 132.17178344726562 , std = 40.89535293766045 , Max = 205.0 , Min = 55.0 , type=float64
 channel0 shape: (128, 192) , mean = -41.404947916666664 , std = 9.420656962306504 , Max = 7 , Min

   olen: 24576 Bytes
   size1: 24588 Bytes
   size2: 15472 Bytes
ratio1= 1.00048828125
ratio2= 0.629557291667
dlen= 24576
mse = 0.0

 channel3 shape: (128, 192) , mean = 26.212158203125 , std = 21.170469863358925 , Max = 89 , Min = -27 , type=int8
   olen: 24576 Bytes
   size1: 24588 Bytes
   size2: 16553 Bytes
ratio1= 1.00048828125
ratio2= 0.673543294271
dlen= 24576
mse = 0.0



16
  t_C shape: (1, 128, 192, 4) , mean = 131.20580037434897 , std = 33.01970345675342 , Max = 200.0 , Min = 66.0 , type=float64
 channel0 shape: (128, 192) , mean = -32.177571614583336 , std = 12.043225029158272 , Max = 1 , Min = -62 , type=int8
   olen: 24576 Bytes
   size1: 24588 Bytes
   size2: 16680 Bytes
ratio1= 1.00048828125
ratio2= 0.6787109375
dlen= 24576
mse = 0.0

 channel1 shape: (128, 192) , mean = 37.389892578125 , std = 14.07385833148873 , Max = 72 , Min = 3 , type=int8
   olen: 24576 Bytes
   size1: 24588 Bytes
   size2: 17309 Bytes
ratio1= 1.00048828125
ratio2= 0.704305013021
dlen= 24576
mse =

dlen= 24576
mse = 0.0



23
  t_C shape: (1, 128, 192, 4) , mean = 130.47940063476562 , std = 27.71016333994277 , Max = 214.0 , Min = 52.0 , type=float64
 channel0 shape: (128, 192) , mean = -24.256306966145832 , std = 15.461586254732813 , Max = 9 , Min = -76 , type=int8
   olen: 24576 Bytes
   size1: 24588 Bytes
   size2: 17249 Bytes
ratio1= 1.00048828125
ratio2= 0.701863606771
dlen= 24576
mse = 0.0

 channel1 shape: (128, 192) , mean = 28.45556640625 , std = 17.70141885363397 , Max = 82 , Min = -1 , type=int8
   olen: 24576 Bytes
   size1: 24588 Bytes
   size2: 17814 Bytes
ratio1= 1.00048828125
ratio2= 0.724853515625
dlen= 24576
mse = 0.0

 channel2 shape: (128, 192) , mean = -17.0211181640625 , std = 11.073150695616588 , Max = 10 , Min = -63 , type=int8
   olen: 24576 Bytes
   size1: 24588 Bytes
   size2: 15867 Bytes
ratio1= 1.00048828125
ratio2= 0.645629882812
dlen= 24576
mse = 0.0

 channel3 shape: (128, 192) , mean = 22.739461263020832 , std = 14.70025594470963 , Max = 86 , Min =

In [46]:
for idx in range(nf):
    print(idx+1)
    
    ratio_1 = np.mean(size1s[idx,:])/np.mean(olens[idx,:])
    ratio_2 = np.mean(size2s[idx,:])/np.mean(olens[idx,:])

    #print('   size_orig= ' + str(size_orig))
    print('   ratio_1= ' + str(ratio_1))
    print('   ratio_2= ' + str(ratio_2))
    print(' ')

1
   ratio_1= 1.00048828125
   ratio_2= 0.68804931640625
 
2
   ratio_1= 1.00048828125
   ratio_2= 0.5120646158854166
 
3
   ratio_1= 1.00048828125
   ratio_2= 0.6423746744791666
 
4
   ratio_1= 1.00048828125
   ratio_2= 0.6877848307291666
 
5
   ratio_1= 1.00048828125
   ratio_2= 0.7214864095052084
 
6
   ratio_1= 1.00048828125
   ratio_2= 0.6952616373697916
 
7
   ratio_1= 1.00048828125
   ratio_2= 0.6645914713541666
 
8
   ratio_1= 1.00048828125
   ratio_2= 0.7358500162760416
 
9
   ratio_1= 1.00048828125
   ratio_2= 0.626922607421875
 
10
   ratio_1= 1.00048828125
   ratio_2= 0.6468302408854166
 
11
   ratio_1= 1.00048828125
   ratio_2= 0.6615702311197916
 
12
   ratio_1= 1.00048828125
   ratio_2= 0.6248575846354166
 
13
   ratio_1= 1.00048828125
   ratio_2= 0.7000935872395834
 
14
   ratio_1= 1.00048828125
   ratio_2= 0.7100728352864584
 
15
   ratio_1= 1.00048828125
   ratio_2= 0.6712646484375
 
16
   ratio_1= 1.00048828125
   ratio_2= 0.6695556640625
 
17
   ratio_1= 1.000488281